In [1]:
import pandas as pd
from pandas import np
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
m1 = pd.read_csv("/content/gdrive/My Drive/finalmerge.csv",index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
X=m1.drop(["DepTime","DepDel15","DepDelayMinutes","ArrDelayMinutes","ArrTime","ArrDel15","Unnamed: 0_x","Unnamed: 0_y","Unnamed: 0.1","Origin","Dest","airport_x","airport_y","FlightDate","date_y","date_x","time_x","time_y"],axis=1)
X

,Quarter,Year,Month,DayofMonth,CRSDepTime,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x,windspeedKmph_y,winddirDegree_y,weatherCode_y,precipMM_y,visibility_y,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y
0,1,2016,1,1,700,14747,12478,1600,22,255,113,0.0,10,1015,0,32,26,41,32,74,7,107,113,0.0,10,1030,0,23,8,34,29,66
1,1,2016,1,1,700,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46,7,107,113,0.0,10,1030,0,23,8,34,29,66
2,1,2016,1,1,700,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46,7,107,113,0.0,10,1030,0,23,8,34,29,66
3,1,2016,1,1,700,14747,12478,1500,21,254,113,0.0,10,1016,0,33,24,42,33,74,7,107,113,0.0,10,1030,0,23,8,34,29,66
4,1,2016,1,1,700,14747,12892,1000,10,39,113,0.0,10,1020,9,29,13,53,49,42,7,107,113,0.0,10,1030,0,23,8,34,29,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815400,3,2017,9,18,2200,12478,12892,200,5,220,113,0.0,10,1012,1,70,6,74,74,87,19,60,122,0.0,10,1018,98,67,26,69,69,92
1815401,3,2017,9,21,2200,12478,12892,200,11,179,143,0.0,10,1010,56,63,12,65,65,91,18,28,122,0.0,10,1015,96,64,26,72,72,75
1815402,3,2017,9,22,2200,12478,12892,200,10,135,113,0.0,10,1010,3,59,12,67,67,75,22,14,116,0.0,10,1014,48,57,33,71,71,63
1815403,3,2017,9,23,2100,12478,12892,100,6,202,113,0.0,10,1009,4,61,8,68,68,80,12,1,113,0.0,10,1017,0,68,15,86,86,55


In [4]:
X.shape

(1815405, 32)

In [0]:
Y=m1["ArrDel15"]

In [6]:
import collections
collections.Counter(Y)

Counter({0.0: 1436039, 1.0: 379366})

In [0]:
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = None) 

In [8]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [9]:
sm = SMOTE()
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [10]:
import collections
collections.Counter(Y_train_res)

Counter({0.0: 1149088, 1.0: 1149088})

In [15]:
#EXTRATREESCLASSIFIER

from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_jobs=-1,random_state = None)
etc.fit(X_train_res,Y_train_res)



ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
Y_predetc=etc.predict(X_test)

In [18]:
#ACCURACY
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_predetc)
#accuracy_score(Y_train,Y_predetc)

0.7877498409445827

In [19]:
from sklearn import metrics
cm=metrics.confusion_matrix(Y_test,Y_predetc)

print(cm)

[[257815  29136]
 [ 47928  28202]]


In [20]:
from sklearn.metrics import classification_report
target_names=["Class 0","Class 1"]
print(classification_report(Y_test,Y_predetc,target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.84      0.90      0.87    286951
     Class 1       0.49      0.37      0.42     76130

    accuracy                           0.79    363081
   macro avg       0.67      0.63      0.65    363081
weighted avg       0.77      0.79      0.78    363081



In [25]:
#DECISION TREE
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()
clf.fit(X_train_res, Y_train_res)#DECISION TREE


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
Y_preddec=clf.predict(X_test)

In [27]:
from sklearn import metrics
cm=metrics.confusion_matrix(Y_test,Y_preddec)
print(cm)

[[239146  47805]
 [ 47258  28872]]


In [28]:
target_names=["Class 0","Class 1"]
print(classification_report(Y_test,Y_preddec,target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.83      0.83      0.83    286951
     Class 1       0.38      0.38      0.38     76130

    accuracy                           0.74    363081
   macro avg       0.61      0.61      0.61    363081
weighted avg       0.74      0.74      0.74    363081



In [29]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_res,Y_train_res)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
Y_predgau=gnb.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_predgau)

0.683610544203635

In [32]:
#XGBoost
import xgboost 
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train_res, Y_train_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
X_test_np = X_test.values

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_predgau)

0.683610544203635

In [0]:
X_total = X.values

In [0]:
Y_total_xgb = model.predict(X_total)

In [0]:
X_new = X.copy()

In [0]:
X_new['NewArr15'] = Y_total_xgb

In [40]:
X_new.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1815365,1815366,1815367,1815368,1815369,1815370,1815371,1815372,1815373,1815374,1815375,1815376,1815377,1815378,1815379,1815380,1815381,1815382,1815383,1815384,1815385,1815386,1815387,1815388,1815389,1815390,1815391,1815392,1815393,1815394,1815395,1815396,1815397,1815398,1815399,1815400,1815401,1815402,1815403,1815404
Quarter,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
Year,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,...,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0
Month,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
DayofMonth,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,21.0,22.0,23.0,25.0,26.0,27.0,28.0,30.0,16.0,16.0,23.0,23.0,12.0,8.0,8.0,2.0,1.0,2.0,2.0,3.0,3.0,4.0,6.0,7.0,8.0,9.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,21.0,22.0,23.0,24.0
CRSDepTime,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,...,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,1800.0,1800.0,1800.0,1800.0,100.0,1800.0,600.0,2200.0,2300.0,2200.0,2300.0,2200.0,2300.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0,2200.0,2200.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0
OriginAirportID,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,13303.0,13303.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,13930.0,13930.0,13930.0,13930.0,13930.0,13930.0,13930.0,...,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,13303.0,13303.0,13303.0,13303.0,12892.0,13303.0,13303.0,11618.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0
DestAirportID,12478.0,12892.0,12892.0,12478.0,12892.0,10397.0,12889.0,14771.0,14771.0,12266.0,14107.0,12478.0,11057.0,12478.0,12478.0,11298.0,11298.0,11618.0,10397.0,10397.0,12266.0,12478.0,12478.0,13930.0,11298.0,13204.0,14747.0,11618.0,11618.0,11292.0,12266.0,14747.0,14107.0,12478.0,11298.0,14771.0,14771.0,12892.0,12892.0,12266.0,...,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,12478.0,10397.0,12478.0,10397.0,13930.0,10397.0,10397.0,12266.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0
CRSArrTime,1600.0,900.0,900.0,1500.0,1000.0,1500.0,900.0,900.0,900.0,1400.0,1100.0,1600.0,1500.0,1600.0,1600.0,15

In [0]:
X_new['ArrDelayMinutes'] = m1['ArrDelayMinutes']

In [44]:
X_new.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1815365,1815366,1815367,1815368,1815369,1815370,1815371,1815372,1815373,1815374,1815375,1815376,1815377,1815378,1815379,1815380,1815381,1815382,1815383,1815384,1815385,1815386,1815387,1815388,1815389,1815390,1815391,1815392,1815393,1815394,1815395,1815396,1815397,1815398,1815399,1815400,1815401,1815402,1815403,1815404
Quarter,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
Year,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,...,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0
Month,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
DayofMonth,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,21.0,22.0,23.0,25.0,26.0,27.0,28.0,30.0,16.0,16.0,23.0,23.0,12.0,8.0,8.0,2.0,1.0,2.0,2.0,3.0,3.0,4.0,6.0,7.0,8.0,9.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,21.0,22.0,23.0,24.0
CRSDepTime,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,...,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,1800.0,1800.0,1800.0,1800.0,100.0,1800.0,600.0,2200.0,2300.0,2200.0,2300.0,2200.0,2300.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0,2200.0,2200.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0,2200.0,2200.0,2200.0,2100.0,2200.0
OriginAirportID,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,14747.0,13303.0,13303.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,13204.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,13930.0,13930.0,13930.0,13930.0,13930.0,13930.0,13930.0,...,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,10397.0,13303.0,13303.0,13303.0,13303.0,12892.0,13303.0,13303.0,11618.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0,12478.0
DestAirportID,12478.0,12892.0,12892.0,12478.0,12892.0,10397.0,12889.0,14771.0,14771.0,12266.0,14107.0,12478.0,11057.0,12478.0,12478.0,11298.0,11298.0,11618.0,10397.0,10397.0,12266.0,12478.0,12478.0,13930.0,11298.0,13204.0,14747.0,11618.0,11618.0,11292.0,12266.0,14747.0,14107.0,12478.0,11298.0,14771.0,14771.0,12892.0,12892.0,12266.0,...,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,11057.0,12478.0,10397.0,12478.0,10397.0,13930.0,10397.0,10397.0,12266.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0,12892.0
CRSArrTime,1600.0,900.0,900.0,1500.0,1000.0,1500.0,900.0,900.0,900.0,1400.0,1100.0,1600.0,1500.0,1600.0,1600.0,15

In [0]:
X_new.to_csv("module3manipulation.csv")